Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/classification-with-onnx/auto-ml-classification-with-onnx.png)

# Quiz - MNIST ONNX Model Registration and Deployment
_**Register best MNIST model trained by AutoML and deploy it for inferencing**_

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Results](#Results)
1. [Test](#Test)



## Introduction

In this example we use the scikit-learn's [digit dataset](http://scikit-learn.org/stable/datasets/index.html#optical-recognition-of-handwritten-digits-dataset) to showcase how you can use AutoML for a simple classification problem and register the best trained model then deploy it for inferencing.

Make sure you have executed the [configuration](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) before running this notebook.

Please find the ONNX related documentations [here](https://github.com/onnx/onnx).

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Configure AutoML using `AutoMLConfig`.
3. Train the model using local compute with ONNX compatible config on.
4. Explore the results and save the ONNX model.
5. Register the saved ONNX model into AML. You can skip step 3 and step 4 if you have the model ready.
6. 

## Setup

As part of the setup you have already created an Azure ML `Workspace` object. For AutoML you will need to create an `Experiment` object, which is a named object in a `Workspace` used to run experiments.

In [ ]:
import logging
import os

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig, constants

In [ ]:
ws = Workspace.from_config()

# Choose a name for the experiment and specify the project folder.
experiment_name = 'automl-classification-onnx'
project_folder = './sample_projects/automl-classification-onnx'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

## Prepare MNIST dataset
In order to train on the MNIST dataset we will first need to download it from Yan LeCun's web site directly and save them in a `data` folder locally.

In [ ]:
import os
import urllib
import numpy as np
import matplotlib.pyplot as plt

#
# Download if the dataset does not exist on local machine.
#

if not os.path.exists('./data/mnist'):
    os.makedirs('./data/mnist', exist_ok=True)
    urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', filename = './data/mnist/train-images.gz')
    urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', filename = './data/mnist/train-labels.gz')
    urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', filename = './data/mnist/test-images.gz')
    urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', filename = './data/mnist/test-labels.gz')
else:
    print("MNIST dataset exists on local machine. Skip downloading.")

#
# Load the dataset.
#

from src.utils import load_data

# Note we also shrink the intensity values (X) from 0-255 to 0-1.
X_train = load_data('./data/mnist/train-images.gz', False) / 255.0
y_train = load_data('./data/mnist/train-labels.gz', True).reshape(-1)

X_test = load_data('./data/mnist/test-images.gz', False) / 255.0
y_test = load_data('./data/mnist/test-labels.gz', True).reshape(-1)

#
# Show some sample images.
#

count = 0
sample_size = 30
plt.figure(figsize = (16, 6))
for i in np.random.permutation(X_train.shape[0])[:sample_size]:
    count = count + 1
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x = 10, y = -10, s = y_train[i], fontsize = 18)
    plt.imshow(X_train[i].reshape(28, 28), cmap = plt.cm.Greys)
plt.show()

#
# Convert X_train and X_test to pandas DataFrame and set column names,
# This is needed for initializing the input variable names of ONNX model, 
# and the prediction with the ONNX model using the inference helper.
#

columns = []
for i in range(X_train.shape[1]):
    columns.append("C" + str(i + 1).zfill(3))
X_train_df = pd.DataFrame(X_train, columns=columns)
X_test_df = pd.DataFrame(X_test, columns=columns)

## Train with enable ONNX compatible models config on

Instantiate an `AutoMLConfig` object to specify the settings and data used to run the experiment.

Set the parameter enable_onnx_compatible_models=True, if you also want to generate the ONNX compatible models. Please note, the forecasting task and TensorFlow models are not ONNX compatible yet.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. Classification supports the following primary metrics: <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>average_precision_score_weighted</i><br><i>norm_macro_recall</i><br><i>precision_score_weighted</i>|
|**iteration_timeout_minutes**|Time limit in minutes for each iteration.|
|**iterations**|Number of iterations. In each iteration AutoML trains a specific pipeline with the data.|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], Multi-class targets.|
|**enable_onnx_compatible_models**|Enable the ONNX compatible models in the experiment.|
|**path**|Relative path to the project folder. AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder.|

In [ ]:
automl_config = AutoMLConfig(task = 'classification',
                             debug_log = os.path.join(project_folder, 'automl_errors.log'),
                             primary_metric = 'AUC_weighted',
                             iteration_timeout_minutes = 60,
                             iterations = 5,
                             verbosity = logging.INFO,
                             X = X_train_df,
                             y = y_train,
                             preprocess=True,
                             enable_onnx_compatible_models=True,
                             path = project_folder)

Call the `submit` method on the experiment object and pass the run configuration. Execution of local runs is synchronous. Depending on the data and the number of iterations this can run for a while.
In this example, we specify `show_output = True` to print currently running iterations to the console.

In [ ]:
local_run = experiment.submit(automl_config, show_output = True)

In [ ]:
local_run

## Results

#### Widget for Monitoring Runs

The widget will first report a "loading" status while running the first iteration. After completing the first iteration, an auto-updating graph and table will be shown. The widget will refresh once per minute, so you should see the graph update as child runs complete.

**Note:** The widget displays a link at the bottom. Use this link to open a web interface to explore the individual run details.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show() 

### Retrieve the best ONNX model

Below we select the best pipeline from our iterations. The `get_output` method returns the best run and the fitted model. The Model includes the pipeline and any pre-processing.  Overloads on `get_output` allow you to retrieve the best run and fitted model for *any* logged metric or for a particular *iteration*.

Set the parameter return_onnx_model=True to retrieve the best ONNX model, instead of the Python model.

In [ ]:
best_run, onnx_mdl = local_run.get_output(return_onnx_model=True)

### Save the best ONNX model

In [ ]:
from azureml.automl.core.onnx_convert import OnnxConverter
onnx_fl_path = os.path.join(project_folder, "best_model.onnx")
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)

### Register your model with Azure ML

In [ ]:
model_dir = project_folder # replace this with the location of your model files

from azureml.core.model import Model

model = Model.register(workspace = ws,
                       model_path = model_dir + "/" + "best_model.onnx",
                       model_name = "mnist_best_onnx",
                       tags = {"onnx": "demo"},
                       description = "MNIST image classification from AutoML",)

### Verify the registered model is working

In [ ]:
import numpy as np
import onnxruntime
from azureml.core.model import Model

#session = onnxruntime.InferenceSession(model_dir + "/best_model.onnx", None)
session = onnxruntime.InferenceSession(Model.get_model_path(model_name = 'mnist_best_onnx'), None)

print(Model.get_model_path(model_name = 'mnist_best_onnx'))

k = 66
X_test_k = X_test[k].tolist()
y_test_k = y_test[k]

print("Ground truth:", y_test_k)

session_input = {}
for x in session.get_inputs():
    i = int(x.name[1:]) - 1 # x.name should be 'CXXX', in which 'XXX' indicates the column index starting from 1.
    session_input[x.name] = np.array([X_test_k[i]]).astype('float32')

r = session.run([session.get_outputs()[0].name], session_input)

print("Prediction:  ", r[0][0])

### Specify our score and environment files

We are now going to deploy our ONNX Model on AML with inference in ONNX Runtime. We begin by writing a score.py file, which will help us run the model in our Azure ML virtual machine (VM), and then specify our environment by writing a yml file. You will also notice that we import the onnxruntime library to do runtime inference on our ONNX models (passing in input and evaluating out model's predicted output). More information on the API and commands can be found in the [ONNX Runtime documentation](https://aka.ms/onnxruntime).

### Write score file

A score file is what tells our Azure cloud service what to do. After initializing our model using azureml.core.model, we start an ONNX Runtime inference session to evaluate the data passed in on our function calls.

In [ ]:
%%writefile score.py
import json
import numpy as np
import onnxruntime
import sys
import os
from azureml.core.model import Model
import time


def init():
    global session, input_name, output_name
    model = Model.get_model_path(model_name = 'mnist_best_onnx')
    session = onnxruntime.InferenceSession(model, None)


def run(input_data):

    try:
        # load in our data, convert to readable format
        data = np.array(json.loads(input_data)['data']).astype('float32').tolist()
        session_input = {}
        for x in session.get_inputs():
            i = int(x.name[1:]) - 1 # x.name should be 'CXXX', in which 'XXX' indicates the column index starting from 1.
            session_input[x.name] = np.array([data[i]]).astype('float32')

        # start timer
        start = time.time()

        r = session.run([session.get_outputs()[0].name], session_input)

        #end timer
        end = time.time()

        result_dict = {"result": int(r[0][0]),
                      "time_in_sec": end - start}
    except Exception as e:
        result_dict = {"error": str(e)}

    return result_dict

### Write environment file

This step creates a YAML environment file that specifies which dependencies we would like to see in our Linux Virtual Machine.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(pip_packages=["numpy", "onnxruntime", "azureml-core"])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

### Write Dockerfile

In [ ]:
%%writefile Dockerfile
RUN apt-get update
RUN apt-get install -y libgomp1

### Create the Container Image
This step will likely take a few minutes.

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  docker_file = "Dockerfile",
                                                  description = "MNIST ONNX Runtime container",
                                                  tags = {"demo": "onnx"}) 


image = ContainerImage.create(name = "onnximage",
                              # this is the model object
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

In case you need to debug your code, the next line of code accesses the log file.

In [ ]:
print(image.image_build_log_uri)

We're all done specifying what we want our virtual machine to do. Let's configure and deploy our container image.

### Deploy the container image

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'demo': 'onnx'}, 
                                               description = 'ONNX for mnist model')

The following cell will likely take a few minutes to run as well.

In [ ]:
from azureml.core.webservice import Webservice

aci_service_name = 'onnx-demo-mnist'
print("Service", aci_service_name)

aci_service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)

aci_service.wait_for_deployment(True)
print(aci_service.state)

In [ ]:
if aci_service.state != 'Healthy':
    # run this command for debugging.
    print(aci_service.get_logs())

# If your deployment fails, make sure to delete your aci_service or rename your service before trying again!
# aci_service.delete()

### Success!

If you've made it this far, you've deployed a working VM with a handwritten digit classifier running in the cloud using Azure ML. Congratulations!

You can get the URL for the webservice with the code below. Let's now see how well our model deals with our test images.

In [ ]:
print(aci_service.scoring_uri)

### Show some sample images in X_test
We use `matplotlib` to plot 10 test images from the dataset.

In [ ]:
plt.figure(figsize = (16, 6))
for k in np.arange(10):
    plt.subplot(1, 15, k+1)
    plt.axhline('')
    plt.axvline('')
    plt.imshow(X_test[k].reshape(28, 28), cmap = plt.cm.Greys)
plt.show()

### Run evaluation / prediction

In [ ]:
plt.figure(figsize = (16, 6), frameon=False)
plt.subplot(1, 11, 1)

plt.text(x = 0, y = -30, s = "True Label: ", fontsize = 11, color = 'black')
plt.text(x = 0, y = -20, s = "Result: ", fontsize = 11, color = 'black')
plt.text(x = 0, y = -10, s = "Inference Time: ", fontsize = 11, color = 'black')
plt.text(x = 3, y = 14, s = "Model Input", fontsize = 10, color = 'black')
plt.text(x = 6, y = 18, s = "(28 x 28)", fontsize = 10, color = 'black')
plt.imshow(np.ones((28,28)), cmap=plt.cm.Greys)    


for i in np.arange(10):
    
    input_data = json.dumps({'data': X_test[i].tolist()})
    
    
    # predict using the deployed model
    r = aci_service.run(input_data)
    
    if "error" in r:
        print("Error:", r['error'])
        break
        
    result = r['result']
    time_ms = np.round(r['time_in_sec'] * 1000, 2)
    
    ground_truth = int(y_test[i])
    
    # compare actual value vs. the predicted values:
    plt.subplot(1, 11, i+2)
    plt.axhline('')
    plt.axvline('')

    # use different color for misclassified sample
    font_color = 'red' if ground_truth != result else 'black'
    clr_map = plt.cm.gray if ground_truth != result else plt.cm.Greys

    # ground truth labels are in blue
    plt.text(x = 10, y = -30, s = ground_truth, fontsize = 18, color = 'blue')
    
    # predictions are in black if correct, red if incorrect
    plt.text(x = 10, y = -20, s = result, fontsize = 18, color = font_color)
    plt.text(x = 5, y = -10, s = str(time_ms) + ' ms', fontsize = 14, color = font_color)

    
    plt.imshow(X_test[i].reshape(28, 28), cmap = clr_map)

plt.show()

### Try classifying your own images!

Create your own handwritten image and pass it into the model.

In [ ]:
# Preprocessing functions take your image and format it so it can be passed
# as input into our ONNX model

import cv2

def rgb2gray(rgb):
    """Convert the input image into grayscale"""
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def resize_img(img_to_resize):
    """Resize image to MNIST model input dimensions"""
    r_img = cv2.resize(img_to_resize, dsize=(28, 28), interpolation=cv2.INTER_AREA)
    r_img.resize((1, 1, 28, 28))
    return r_img

def preprocess(img_to_preprocess):
    """Resize input images and convert them to grayscale."""
    if img_to_preprocess.shape == (28, 28):
        img_to_preprocess.resize((1, 1, 28, 28))
        return img_to_preprocess
    
    grayscale = rgb2gray(img_to_preprocess)
    processed_img = resize_img(grayscale)
    return processed_img

In [ ]:
# Replace this string with your own path/test image
# Make sure your image is square and the dimensions are equal (i.e. 100 * 100 pixels or 28 * 28 pixels)

# Any PNG or JPG image file should work

your_test_image = "<path to file>"

# e.g. your_test_image = "C:/Users/vinitra.swamy/Pictures/handwritten_digit.png"

import matplotlib.image as mpimg

if your_test_image != "<path to file>":
    img = mpimg.imread(your_test_image)
    plt.subplot(1,3,1)
    plt.imshow(img, cmap = plt.cm.Greys)
    print("Old Dimensions: ", img.shape)
    img = preprocess(img)
    print("New Dimensions: ", img.shape)
else:
    img = None

In [ ]:
if img is None:
    print("Add the path for your image data.")
else:
    input_data = json.dumps({'data': img.tolist()})

    try:
        r = aci_service.run(input_data)
        result = r['result']
        time_ms = np.round(r['time_in_sec'] * 1000, 2)
    except KeyError as e:
        print(str(e))

    plt.figure(figsize = (16, 6))
    plt.subplot(1, 15,1)
    plt.axhline('')
    plt.axvline('')
    plt.text(x = -100, y = -20, s = "Model prediction: ", fontsize = 14)
    plt.text(x = -100, y = -10, s = "Inference time: ", fontsize = 14)
    plt.text(x = 0, y = -20, s = str(result), fontsize = 14)
    plt.text(x = 0, y = -10, s = str(time_ms) + " ms", fontsize = 14)
    plt.text(x = -100, y = 14, s = "Input image: ", fontsize = 14)
    plt.imshow(img.reshape(28, 28), cmap = plt.cm.gray)    

In [ ]:
# remember to delete your service after you are done using it!

aci_service.delete()

## Conclusion

Congratulations!

In this tutorial, you have:
- familiarized yourself with ONNX Runtime inference and the pretrained models in the ONNX model zoo
- understood a state-of-the-art convolutional neural net image classification model (MNIST in ONNX) and deployed it in Azure ML cloud
- ensured that your deep learning model is working perfectly (in the cloud) on test data, and checked it against some of your own!

Next steps:
- Check out another interesting application based on a Microsoft Research computer vision paper that lets you set up a [facial emotion recognition model](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/onnx/onnx-inference-facial-expression-recognition-deploy.ipynb) in the cloud! This tutorial deploys a pre-trained ONNX Computer Vision model in an Azure ML virtual machine.
- Contribute to our [open source ONNX repository on github](http://github.com/onnx/onnx) and/or add to our [ONNX model zoo](http://github.com/onnx/models)